In [ ]:
!python D:/progamming/va/truecase/ru-punctuation-truecase/src/prepare_data.py --data_dir D:/progamming/va/truecase/ru-punctuation-truecase/data/tatoeba_dataset --num_samples -1 --percent_dev 0.2

In [ ]:
from typing import List, Tuple
import pandas as pd
import numpy as np

from transformers import AutoModel, AutoTokenizer

import sys
sys.path.insert(0, 'D:/progamming/va/truecase/ru-punctuation-truecase/src')
from process_text import clean_text, clean_text_3times

In [ ]:
df = pd.read_parquet("/Users/falaputin/Downloads/cleaned_top300k_merged_database.parquet.gzip", engine="pyarrow")
df

Подготовка данных для hugging face transformers

Пример из hugging face

In [ ]:
from pathlib import Path
import re
import numpy as np
from sklearn.model_selection import train_test_split

def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs


In [ ]:
texts, tags = read_wnut('D:\\progamming\\va\\truecase\\ru-punctuation-truecase\\data\\wnut17train.conll')
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [ ]:
def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

In [ ]:
len(train_encodings.tokens(3))

In [ ]:
train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [ ]:
for token, label in zip(train_encodings.tokens(), train_labels[0]):
    print(token, label)

Подготовка собственного датасета

In [ ]:
def clean_text_3times(text: str) -> str:
    text = clean_text(text)
    text = clean_text(text)
    text = clean_text(text)
    return text

def file2array(path_to_file: str) -> List[List[str]]:
    result = []
    with open(path_to_file, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            result.append(line.split(' '))
    return result


def textlabel2arrays(path_to_text: str, path_to_labels: str) -> Tuple[List[List[str]], List[List[str]]]:
    texts = []
    labels = []
    with open(path_to_text, 'r') as f_text:
        with open(path_to_labels, 'r') as f_labels:
            for line_text, line_labels in zip(f_text.readlines(), f_labels.readlines()):

                line_text = line_text.strip()
                line_labels = line_labels.strip()
                
                texts.append(line_text.split(' '))
                labels.append(line_labels.split(' '))
    return texts, labels


def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

In [ ]:
path_to_train_text = "../data/tatoeba_dataset/text_train.txt"
path_to_train_labels = "../data/tatoeba_dataset/labels_train.txt"
path_to_val_text = "../data/tatoeba_dataset/text_dev.txt"
path_to_val_labels = "../data/tatoeba_dataset/labels_dev.txt"

train_texts, train_tags = textlabel2arrays(path_to_train_text, path_to_train_labels)
val_texts, val_tags = textlabel2arrays(path_to_val_text, path_to_val_labels)

unique_tags = set(tag for doc in train_tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

print(len(train_texts) == len(train_tags))
print(len(val_texts) == len(val_tags))
print(len(train_texts), len(train_tags))
print(len(val_texts), len(val_tags))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-conversational', model_max_length=512)
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [ ]:
s = ' '.join(val_texts[99989])
s

In [ ]:
def remove_unicode_chars(input_str):
    string_encode = input_str.encode("ascii", "ignore")
    string_decode = string_encode.decode()
    return string_decode

s = ' '.join(['\u200bникто', 'не', 'знал', 'настоящую', 'глубину', 'озера'])
# s = ' '.join(val_texts[36934])
# s = 'почему дзюдо'
remove_unicode_chars('\u200bникто')

In [ ]:
import re

def remove_unicode_chars(input_str):
    pattern = re.compile(u"[\u200c-\u200f\u202a-\u202f\u2066-\u2069-\u200b]")
    input_str = pattern.sub("", input_str)
    input_str = input_str.replace('\xad', '')
    input_str = input_str.replace('  ', ' ')
    return input_str.strip()

s = ' '.join(['почему', 'дзюдо', '\xad', 'лучший', 'выбор', 'для', 'вашего', 'ребёнка'])
remove_unicode_chars(s)

In [ ]:
s = 'почему дзюдо \xad лучший выбор для вашего ребёнка'
 
#encode() method
strencode = s.encode("ascii", "ignore")
 
#decode() method
strdecode = strencode.decode()
strdecode

In [ ]:
s = "почему дзюдо ­ лучший выбор для вашего ребёнка"
s = ' '.join(['\u200bникто', 'не', 'знал', 'настоящую', 'глубину', 'озера'])
clean_text(s)

In [ ]:
labels = [[tag2id[tag] for tag in doc] for doc in val_tags]
encoded_labels = []
idx = 0
for doc_labels, doc_offset in zip(labels, val_encodings.offset_mapping):
    # create an empty array of -100
    doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
    arr_offset = np.array(doc_offset)
    # print(doc_enc_labels)
    # print(len(arr_offset))

    # set labels whose first offset position is 0 and the second is not 0
    try:
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())
    except Exception as e:
        print(idx)
        
    idx += 1

In [ ]:
'set labels whose first offset position is 0 and the second is not 0'.split()

In [ ]:
broken_texts_128 = [36934, 80799, 80800, 99989, 119863, 126620, 130799, 131430, 139275, 160289]
broken_texts_512 = [126620, 130799, 160289]
for idx in broken_texts_128:
    print(val_texts[idx])
    print(len(val_texts[idx]))

In [ ]:
train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [ ]:
for token, label in zip(train_encodings.tokens(), train_labels[0]):
    print(token, label)